In [1]:
leads = ["I", "II", "III", "aVR", "aVL", "aVF", "V1", "V2", "V3", "V4", "V5", "V6"]

## Gamma, a (12,5) matrix to modify the five waves' amplitudes of 12 leads (P, Q, R, S, T)
gamma = np.array(
    [
        [1, 0.1, 1, 1.2, 1],
        [2, 0.2, 0.2, 0.2, 3],
        [1, -0.1, -0.8, -1.1, 2.5],
        [-1, -0.05, -0.8, -0.5, -1.2],
        [0.05, 0.05, 1, 1, 1],
        [1, -0.05, -0.1, -0.1, 3],
        [-0.5, 0.05, 0.2, 0.5, 1],
        [0.05, 0.05, 1.3, 2.5, 2],
        [1, 0.05, 1, 2, 1],
        [1.2, 0.05, 1, 2, 2],
        [1.5, 0.1, 0.8, 1, 2],
        [1.8, 0.05, 0.5, 0.1, 2],
    ]
)

# heart rate
mu_hr_1 = heart_rate  # mean of the heart rate
sigma_hr_1 = 7  # variance of the heart rate

# noise
min_noise_1 = 0.01
max_noise_1 = 0.1

# For PQRST five spikes
# t, the starting position along the circle of each interval in radius
mu_t_1 = np.array((-70, -15, 0, 15, 100))
sigma_t_1 = np.ones(5) * 3

# a, the amplitude of each spike; b, the width of each spike
mu_a_1 = np.array((1.2, -5, 30, -7.5, 0.75))
mu_b_1 = np.array((0.25, 0.1, 0.1, 0.1, 0.4))
sigma_a_1 = np.abs(mu_a_1 / 5)
sigma_b_1 = np.abs(mu_b_1 / 5)

## Abnormal ECG Parameters
# heart rate
mu_hr_2 = 60  # mean of the heart rate
sigma_hr_2 = 7  # variance of the heart rate

# noise
min_noise_2 = 0.01
max_noise_2 = 0.1

# t, a, b
mu_t_2 = np.array((-70, -15, 0, 15, 100))
mu_a_2 = np.array((1.2, -4, 25, -6.5, 0.75))
mu_b_2 = np.array((0.25, 0.1, 0.1, 0.1, 0.4))
sigma_t_2 = np.ones(5) * 3
sigma_a_2 = np.abs(mu_a_1 / 5)
sigma_b_2 = np.abs(mu_b_1 / 5)


def simulation(normal_N, abnormal_N, save_params=False):
    normal_data = []
    normal_params = []
    print("Creating normal dataset")
    for i in range(normal_N):
        ti = np.random.normal(para.mu_t_1, para.sigma_t_1)
        ai = np.random.normal(para.mu_a_1, para.sigma_a_1)
        bi = np.random.normal(para.mu_b_1, para.sigma_b_1)
        hr = np.random.normal(para.mu_hr_1, para.sigma_hr_1)
        noise = np.random.uniform(low=para.min_noise_1, high=para.max_noise_1)
        ecgs, _ = ecg_simulate_multichannel(
            duration=para.duration * 2,
            sampling_rate=para.sampling_rate,
            noise=noise,
            Anoise=para.Anoise,
            heart_rate=hr,
            gamma=para.gamma,
            ti=ti,
            ai=ai,
            bi=bi,
        )
        ecgs = np.array(ecgs)
        start_i = np.random.randint(len(ecgs[0]) // 4, len(ecgs[0]) // 2)
        normal_data.append(ecgs[:, start_i : start_i + para.sampling_rate * para.duration])
        normal_params.append(
            {"ti": ti, "ai": ai, "bi": bi, "hr": hr, "noise": noise, "gamma": para.gamma}
        )

    abnormal_data = []
    abnormal_params = []
    print("Creating abnormal dataset")
    for i in range(abnormal_N):
        ti = np.random.normal(para.mu_t_2, para.sigma_t_2)
        ai = np.random.normal(para.mu_a_2, para.sigma_a_2)
        bi = np.random.normal(para.mu_b_2, para.sigma_b_2)
        hr = np.random.normal(para.mu_hr_2, para.sigma_hr_2)
        noise = np.random.uniform(low=para.min_noise_2, high=para.max_noise_2)
        ecgs, _ = ecg_simulate_multichannel(
            duration=para.duration * 2,
            sampling_rate=para.sampling_rate,
            noise=noise,
            Anoise=para.Anoise,
            heart_rate=hr,
            gamma=para.gamma,
            ti=ti,
            ai=ai,
            bi=bi,
        )
        ecgs = np.array(ecgs)
        start_i = np.random.randint(len(ecgs[0]) // 4, len(ecgs[0]) // 2)
        abnormal_data.append(ecgs[:, start_i : start_i + para.sampling_rate * para.duration])
        abnormal_params.append(
            {"ti": ti, "ai": ai, "bi": bi, "hr": hr, "noise": noise, "gamma": para.gamma}
        )

    labels = np.array([0] * len(normal_data) + [1] * len(abnormal_data))
    permutation = np.random.permutation(len(labels))
    data = np.array(normal_data + abnormal_data)
    data_params = np.array(normal_params + abnormal_params)
    labels = labels[permutation]
    data = data[permutation]
    data_params = data_params[permutation]

    np.save("sim_ecg_data", data)  # save ECG data
    np.save("sim_ecg_labels", labels)  # save label
    if save_params:
        np.save("sim_ecg_params", data_params)  # save parameters for each ecg sample (12,2500)


In [2]:
'''
Function Name: simulation
Input: 
        normal_N: the number of normal ECG data;
        abnormal_N: the number of abnormal ECG data;
        save_params: whether save parameters for each ecg sample, default value is False.
Output:
        'sim_ecg_data.npy': output file, an array of shape (normal_N + abnormal_N, 12, sampling_rate*duration);
        'sim_ecg_labels.npy': file to save labels;
        'sim_ecg_params.npy': depend on save_params, file to save parameters for each ecg sample.
The saved data is already shuffled.

For more parameters' settings, please check 'parameters.py' file.
'''
normal_N = 3
abnormal_N = 3
save_params = False
s.simulation(normal_N,abnormal_N,save_params)

Creating normal dataset
Creating abnormal dataset


In [10]:
import numpy as np
data_shape = np.load('sim_ecg_data.npy').shape
print('data shape: ',data_shape)
labels = np.load('sim_ecg_labels.npy')
print(labels)

data shape:  (6, 12, 2500)
[0 1 0 0 1 1]
